In [ ]:
import pandas as pd #Forgot to add the pd!


In [ ]:
raw_data = pd.read_csv('FIFA_raw_data.csv')

In [ ]:
pd.options.display.max_columns = None
raw_data

In [ ]:
df = raw_data.copy()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df['Goalkeeping'].max()

In [ ]:
pd.options.display.max_rows = None
df['Goalkeeping']

In [ ]:
df[(df['Goalkeeping'] > 99)]

In [ ]:
df[(df['Goalkeeping'] > 99) & ['Best Position'] != 'GK')] #This is causing it a crash =/

In [ ]:
df = df.drop(['ID', 'LongName', 'photoUrl', 'playerUrl'], axis = 1) #Ok, let this be a lesson on dropping things...

#PREMATURELY

In [ ]:
df.head()

In [ ]:
duplicate_column_names = df.columns[df.columns.duplicated()].tolist()

if duplicate_column_names:
    print(f"Duplicate column names: {duplicate_column_names}")
else:
    print("No duplicate columns")

In [ ]:
df.columns.values #Going here to rename OVA

In [ ]:
df = df.rename(columns={'↓OVA':'OVA'}) #Cool easier way to rename

In [ ]:
df.head()

# Checkpoint

Doing some further tidying an exploration above

Below will be dropping more columns and tidying them properly

# Sorting Club

Removing the weird n's from in front of the club names

In [ ]:
df['Club'].head() #Can see that there are 4 n's for each one

In [ ]:
df["Club"]=df["Club"].str.strip("\r\n\r\n\r\n\r\n") #Have to add the double backslash at the end otherwise it interprets it \
#As an escape character still. Not sure why...

In [ ]:
df["Club"]=df["Club"].str.strip("\n\n\n\n") #Have to add the double backslash at the end otherwise it interprets it \
#As an escape character still. Nopt sure why...
# Removed the r to fix it, because it's not a raw string, it's actual new lines in the data!!!

- `"\r\n\r\n\r\n\r\n"` - Interpreted as 8 actual control characters (4 carriage returns and 4 newlines)
- `r"\n\n\n\n"` - Would be interpreted as 8 literal characters (the backslashes and letters)

The reason this doesn't work is that \n is Pys way of showing a new line. So while on the output it appears as \n, the data isn't actually that. The data is new line, fixed above by not treating it as a raw string, and just having "\n\n..." as the string to remove!


In [ ]:
df['Club'].head() #Success!

In [ ]:
df.head()

In [ ]:
type(df['Age'][0])

In [ ]:
type(df['Value'][0])

In [ ]:
type(df['Height'][0])

In [ ]:
df['Best Position'].unique()

Attempting to fix the columns that should be numerical

In [ ]:
df_numerical = df.copy()

# Height

Removing cm is easy, but check unique values after and you'll see some are recorded in ft and inches

In [ ]:
df['Height'] = df['Height'].str.replace('cm', '')

In [ ]:
#Other way to do this with strip

df['Height'] = df['Height'].str.strip("cm")

In [ ]:
df['Height'].head()

In [ ]:
df['Height'].unique()

In [ ]:
#Copied this straight from online

def ft_to_cm(x):
    if "'" in x: #If it finds a ft marker
        ft=x.replace("'",".").replace('"',"") #New var ft is height, turned to e.g. 6'3" -> 6.3
        cm= round(float(ft)*30.48,0)
        return int(cm)

    else:
        return int(x)

df["Height"]=df["Height"].apply(ft_to_cm)
df["Height"].unique()

#Issue I'm noting is that this wouln't work with someone, say 5'11"

In [ ]:
def ft_to_cm(x):
    if "'" in x:
        # Split the string into feet and inches parts
        parts = x.replace('"', '').split("'")
        feet = float(parts[0])
        # Check if there are inches specified
        inches = float(parts[1]) if parts[1] else 0
        
        # Convert to centimeters (1 foot = 30.48 cm, 1 inch = 2.54 cm)
        cm = round((feet * 30.48) + (inches * 2.54), 0)
        return int(cm)
    else:
        return int(x)

df["Height"]=df["Height"].apply(ft_to_cm)
df["Height"].unique()

# Weight

As above, we still have differing scales of measurement, so let's fix that...

In [ ]:
df['Weight'] = df['Weight'].str.replace('kg', '')

#Or, as above, the alternate way to do it

In [ ]:
df['Weight'].head()

In [ ]:
df['Weight'].unique()

In [ ]:
type(df['Weight'][0])

In [ ]:
def lbs_to_kg(x):
    if "lbs" in x:
        lbs = x.replace("lbs", "")
        kg = round(int(lbs) / 2.2, 0)
        return int(kg)
    else:
        return int(x)

#Forgot to actually apply the damn thing

df['Weight'] = df['Weight'].apply(lbs_to_kg)

#What didn't work was that these are strings, need to convert everything to integers, including the returned x's, to get a nice
#clean set of numbers

In [ ]:
df['Weight'].unique()

In [ ]:
df.rename(columns={'Height':'Height(cm)','Weight':'Weight(kg)'},inplace =True)

#`inplace=True` - This parameter tells pandas to modify the original DataFrame directly instead of returning a new copy. The changes will be applied to `df` itself rather than requiring reassignment.

# Fixing Value

Checkpoint below

df['Value'] = df['Value'].str.replace('M', '')
df['Value'] = df['Value'].str.replace('€', '')
df['Value'] = df['Value'].str.replace('.', '')

Not using this as there's a better way

In [ ]:
df_value = df.copy()

In [ ]:
df[['Value', 'Wage']].head() #Nice way to apply two .heads

In [ ]:
def money(x):
    if "€" in x:
        x = x.replace("€", "")

    if "M" in x: #Wonder a way to do this case insensitively...
        x=x.replace("M", "")
        return int(float(x) * 1000000) #Can we do this so it automatically returns it in the mil value... 
# The intfloat bit works because it floats a value, say 103.5, then returns the integer (the decimal is eliminated by multiplying it)
    
    elif "K" in x:
        x = x.replace("K", "")
        return int(float(x) * 1000)

    return int(x)

# Ok, they've gone for displaying value in mils and wage in ks, I think I wanna keep them as pure monetary numbers

df['Value'] = df['Value'].apply(money)
df['Wage'] = df['Wage'].apply(money)

In [ ]:
df.rename(columns = {'Value':'Value (€)', "Wage":"Wage (€)"})

df[['Value', 'Wage']].head()

# Checking in again

A quick review and checkpoint to see what' next
Forgot the checkpoint

In [ ]:
df.head()

Missed release clause, but was planning to drop that anyway... but oh well, let's go back and do it
W/F
SM
IR
all have a star at the end, let's remove that and I think we're done!

In [ ]:
df['Release Clause'] = df['Release Clause'].apply(money)
df['Release Clause'].head()

In [ ]:
df[['W/F', 'SM', 'IR']].head()

In [ ]:
df['W/F'] = df['W/F'].str.replace("★", "")
df['SM'] = df['SM'].str.replace("★", "")
df['IR'] = df['IR'].str.replace("★", "")

In [ ]:
df[['W/F', 'SM', 'IR']].head()

In [ ]:
df_nearly = df.copy()

In [ ]:
df.rename(columns = {'W/F':'Weak Foot /5', "SM":"Skill Moves /5", "IR": "International Reputation /5"}, inplace = True)

In [ ]:
df.head()

# Contract

Have still got it in, so for practice's sake, can I convert it

In [ ]:
df_contract = df.copy()

In [ ]:
df['Contract'].head()

In [ ]:
df['Contract'].unique()

So.. many are years separated with ~, some have On Loan, and show MMM (D)D, YYYY
Very nearly missed 'Free'

So, we have three types of contract 'Free', On loan, and regular OR 'Free', 'Loan', 'Contract'

In [ ]:
help(pd.na)

In [ ]:
def type(x):
    if "Free" in x:
        return "Free"
    if "Loan" in x:
        return "Loan"
    if "~" in x:
        return "Contract"
    else:
        return pd.na

def contract_start(x):
    if "~" in x:
        return pd.to_datetime(x[:4])
    if "Loan" in x:
        x = x.str.strip(" On Loan")
        x = 

# Abandoning this at this point because of the whole datetime module that I don't understand. Will copypaste and try to understand the correct code beloq

In [ ]:
df['Loan Date End'].unique()

In [ ]:
from datetime import datetime

def type(x):
    if "Free" in x:
        return "Free"
    if "Loan" in x:
        return "Loan"
    if "~" in x:
        return "Contract"
    else:
        return pd.na

def start_time_contract(x):
    if "~" in x:
        return  int(x[:4]) #Kept as an integer, reasons below
    if "Loan" in x:
        x=x.strip(" On Loan")
        x= datetime.strptime(x, "%b %d, %Y")
        return x.date()
    else:
        return pd.NA

def end_time_contract(type,contract,loan):
    if type == "Contract":
        return int(contract[-4:])
    if type == "Loan":
        loan = datetime.strptime(loan, "%b %d, %Y")
        return loan.date()

    else:
        return pd.NA

df["Type of contract"] = df["Contract"].apply(type)
df["Start year"] = df["Contract"].apply(start_time_contract)
df["End year"] = df.apply(lambda row: end_time_contract(row["Type of contract"],row["Contract"],row["Loan Date End"] ), axis=1)

# Explaining the above

type(x): I wrote this myself, so this is fine to understand. It just categorises the type of contract based on keywords

start_time_contract: Is extracting the start date from each value. Year is kept as an integer for the following reasons:

1. **Accuracy**: Creating a date object like `datetime(2020, 1, 1)` implies January 1st, which adds false precision that wasn't in the original data

2. **Simplicity**: Integers are simpler to work with for year-level calculations (e.g., contract duration)

3. **Performance**: Integers use less memory and are faster to process than date objects

4. **Clarity**: Using an integer clearly communicates that only the year is known, not a specific date

5. **Avoiding assumptions**: Date objects require month and day values, forcing you to make assumptions (like January 1st) that weren't in the original data

The loan part first strips loan, then converts the values to datetime using a function from the dt module. Returning loan.date() only includes date info, not time.

The last one specifies what to do and is then returned with the lambda function to make the end year column. The three parameters are the type of contract, which by then is in its own column, the existing contract row, and the loan end date column. 

Lambda creates a function that takes the parameter 'row'... I still don't get it... but row is just the function name. It's doing my head in right now.
